
### Dependencies and Setup


In [ ]:
knitr::opts_chunk$set(echo = TRUE)

#install.packages("ggplot2")
#install.packages("dplyr")
#install.packages("readxl")
#install.packages(stats)
# 
library(ggplot2)
library(dplyr)
library(readxl)
library(stats)


# Importamos Datos:

Los datos importados son:

* Pax_Nal: Pasajeros (Salidas) en vuelos nacionales
* Pax_Int: Pasajeros (Salidas) en vuelos internacionales 
* Vue_Nal: Vuelos u operaciones (Salidas) en vuelos nacionales
* Vue_Int: Vuelos u operaciones (Salidas) en vuelos internacionales
* Pax_Metro: Pasajeros transportados (Millones) en el SCM
* D_Sep2017: Dummy que toma el valor de 1 en septiembre de 2017
* D_Oct2017: Dummy que toma el valor de 1 en octubre de 2017
* D_Feb2020: Dummy que toma el valor de 1 en febrero de 2020
* D_Mar2020: Dummy que toma el valor de 1 en marzo de 2020
* D_Abr2020: Dummy que toma el valor de 1 en abril de 2020
* D_May2020: Dummy que toma el valor de 1 en mayo de 2020
* D_Jun2020: Dummy que toma el valor de 1 en junio de 2020
* D_Jul2020: Dummy que toma el valor de 1 en julio de 2020
* D_Ene2021: Dummy que toma el valor de 1 en enero de 2021
* D_Mar2021: Dummy que toma el valor de 1 en marzo de 2021
* D_Ene: Dummy que toma el valor de 1 en todos los eneros
* D_Dic: Dummy que toma el valor de 1 en todos los diciembres

Fuente: INEGI, https://www.inegi.org.mx/app/indicadores/?tm=0&t=1090


In [ ]:
Datos <- read_excel("Base_Transporte_ARIMA.xlsx", sheet = "Datos", col_names = TRUE)

head(Datos)


# Conversion a series de tiempo:



In [ ]:
Pax_Nal <- ts(Datos$Pax_Nal, 
              start = c(2000, 1), 
              freq = 12)

LPax_Nal <- ts(log(Datos$Pax_Nal), 
               start = c(2000, 1), 
               freq = 12)

DLPax_Nal <- ts(log(Datos$Pax_Nal) - lag(log(Datos$Pax_Nal), k = 1),
                start = c(2000, 1), 
                freq = 12)

D_Mar2020	<- ts(Datos$D_Mar2020, 
                start = c(2000, 1), 
                freq = 12)

D_Abr2020	<- ts(Datos$D_Abr2020, 
                start = c(2000, 1), 
                freq = 12)

D_Jun2020	<- ts(Datos$D_Jun2020, 
                start = c(2000, 1), 
                freq = 12)

D_Jul2020 <- ts(Datos$D_Jul2020, 
                start = c(2000, 1), 
                freq = 12)

D_Mar2021 <- ts(Datos$D_Mar2021, 
                start = c(2000, 1), 
                freq = 12)

D_Ene <- ts(Datos$D_Ene, 
            start = c(2000, 1), 
            freq = 12)

D_Feb <- ts(Datos$D_Feb, 
            start = c(2000, 1), 
            freq = 12)

D_Jul <- ts(Datos$D_Jul, 
            start = c(2000, 1), 
            freq = 12)

D_Dic <- ts(Datos$D_Dic, 
            start = c(2000, 1), 
            freq = 12)


# Graficas:



In [ ]:
par(mfrow=c(3,1))

plot(Pax_Nal, xlab = "Tiempo", ylab = "Pasajeros",
     main = "Pasajeros en vuelos nacionales de salida",
     col = "darkgreen")

plot(LPax_Nal, xlab = "Tiempo", ylab = "LN Pasajeros", 
     main = "LN Pasajeros en vuelos nacionales de salida",
     col = "darkblue")

plot(DLPax_Nal, xlab = "Tiempo",  ylab = "DLN Pasajeros",
     main = "Diff LN Pasajeros en vuelos nacionales de salida", 
     col = "darkred")

par(mfrow=c(1,1))


# Estimacion del proceso ARIMA(p, 1, q):



In [ ]:
ARMA_Ex_DLPax_Nal_2 <- arima(DLPax_Nal, order = c(6, 0, 6),
                             xreg = cbind(D_Ene, D_Feb, D_Jul, D_Dic, D_Mar2020, D_Abr2020, D_Jun2020, D_Jul2020, D_Mar2021),
                             method = "ML")

ARMA_Ex_DLPax_Nal_2


# Forecast:

Datos Exogenos:


In [ ]:
Predict_Datos <- read_excel("Predict_Base_Transporte_ARIMA.xlsx", sheet = "Datos", col_names = TRUE)

head(Predict_Datos)


# Conversion a series de tiempo:



In [ ]:
D_Mar2020_f	<- ts(Predict_Datos$D_Mar2020, 
                start = c(2021, 8), 
                freq = 12)

D_Abr2020_f	<- ts(Predict_Datos$D_Abr2020, 
                start = c(2021, 8), 
                freq = 12)

D_Jun2020_f	<- ts(Predict_Datos$D_Jun2020, 
                start = c(2021, 8), 
                freq = 12)

D_Jul2020_f <- ts(Predict_Datos$D_Jul2020, 
                start = c(2021, 8), 
                freq = 12)

D_Mar2021_f <- ts(Predict_Datos$D_Mar2021, 
                start = c(2021, 8), 
                freq = 12)

D_Ene_f <- ts(Predict_Datos$D_Ene, 
            start = c(2021, 8), 
            freq = 12)

D_Feb_f <- ts(Predict_Datos$D_Feb, 
            start = c(2021, 8), 
            freq = 12)

D_Jul_f <- ts(Predict_Datos$D_Jul, 
            start = c(2021, 8), 
            freq = 12)

D_Dic_f <- ts(Predict_Datos$D_Dic, 
            start = c(2021, 8), 
            freq = 12)


# Prediccion 2 años de la serie en diferencias:



In [ ]:
predict(ARMA_Ex_DLPax_Nal_2, n.ahead = 24, 
        newxreg = cbind(D_Ene_f, D_Feb_f, D_Jul_f, D_Dic_f, 
                        D_Mar2020_f, D_Abr2020_f, D_Jun2020_f, D_Jul2020_f, D_Mar2021_f))

DLPax_Nal_f <- predict(ARMA_Ex_DLPax_Nal_2, n.ahead = 24, 
                      newxreg = cbind(D_Ene_f, D_Feb_f, D_Jul_f, D_Dic_f, 
                                      D_Mar2020_f, D_Abr2020_f, D_Jun2020_f, D_Jul2020_f, D_Mar2021_f))

names(DLPax_Nal_f)


# Prediccion de la serie original:



In [ ]:
Pronostico_Arima <- read_excel("Pronostico_Arima.xlsx", sheet = "Datos", col_names = TRUE)

Pronostico_Arima$Pax_Nal_f <- Pronostico_Arima$Pax_Nal

for(i in 1:24){
  Pronostico_Arima$Pax_Nal_f[259 + i] <- 
    Pronostico_Arima$Pax_Nal_f[259 + i - 1]*(1 + DLPax_Nal_f$pred[i])
}

View(Pronostico_Arima)


# Grafica



In [ ]:
options(scipen = 999) # NO notacion cientifica

ggplot(data = Pronostico_Arima, aes(x = Periodo)) +
  geom_line(aes(y = Pax_Nal_f, color = "Pax_Nal_f")) +
  geom_line(aes(y = Pax_Nal, color = "Pax_Nal")) +
  scale_color_brewer(type = "qual", palette = 2) +
  #theme_bw() + 
  theme(legend.position = "bottom") +
  theme(legend.title = element_blank()) +
  guides(col = guide_legend(nrow = 1, byrow = TRUE)) + 
  xlab("Tiempo") + 
  ylab("Pasajeros") + 
  theme(plot.title = element_text(size = 11, face = "bold", hjust = 0)) + 
  theme(plot.subtitle = element_text(size = 10, hjust = 0)) + 
  theme(plot.caption = element_text(size = 10, hjust = 0)) +
  theme(plot.margin = unit(c(1,1,1,1), "cm")) +
  labs(
    title = "Pasajeros en vuelos nacionales (Salidas)",
    subtitle = "(Ene-2000 a Jun-2019)",
    caption = "Fuente: Elaboración propia"
  )

ggsave("Pax_Nal_f.png", width = 20, height = 15, units = "cm")
